# Draft analysis 

---

Group name: Gruppe D

---


## Introduction

*This section includes an introduction to the project motivation, data, and research question. Include a data dictionary* 

## Setup

In [1]:
import pandas as pd

## Data

## Import data

In [ ]:
# import csvs

# 1. Februarwoche 2019
df_19 = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2019/download/Fahrzeiten_SOLL_IST_20190203_20190209.csv")

# 1. Februarwoche 2021
df_21 = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2021/download/Fahrzeiten_SOLL_IST_20210131_20210206.csv")

# Haltestellen
df_haltestellen = pd.read_csv('https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2019/download/Haltestelle.csv')

# Haltepunkte
df_haltepunkte = pd.read_csv('https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2019/download/Haltepunkt.csv')

# Linien
df_linie = pd.read_csv('https://data.stadt-zuerich.ch/dataset/vbz_fahrgastzahlen_ogd/download/LINIE.csv', delimiter=';')


### Data structure

### Data corrections

## Analysis

### Descriptive statistics

### Exploratory data analysis

## Visualizations

### Visualization ideas

### Save Visualizations



Save your draft visualizations in the folder `reports/visualizations/`. Use a meaningful name (always include the word `draft` and a `timestamp`in your filename).

## Conclusion and recommended action